In [2]:
import matplotlib
import operator
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib

# Set 
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = 10

# import custom functions
from functions_statistical_performance import *

In [3]:
# True to exactly reproduce the results presented in the paper (same classifiers)
reproduceflag=False

In [4]:
# function to predict over a generic db when you already have the classifiers
# it provides the average FPro 
def ClassifyDB(db, model_per_fold,nut_sel):
    dbsel=db.loc[:,nut_sel]
    Xnut=dbsel.values    
    
    indfold=0
    for model in model_per_fold:
        indfold+=1
        y_pred = model.predict(Xnut)
        y_probs= model.predict_proba(Xnut)
        db['classf'+str(indfold)]=y_pred
        db['p1f'+str(indfold)]=y_probs[:,0]
        db['p2f'+str(indfold)]=y_probs[:,1]
        db['p3f'+str(indfold)]=y_probs[:,2]
        db['p4f'+str(indfold)]=y_probs[:,3]
        db['FProf'+str(indfold)]=(1-db['p1f'+str(indfold)]+db['p4f'+str(indfold)])/2
     
    for p in range(1,5):
        db['p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].mean(axis=1)
        db['std_p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].std(axis=1)


    db['FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5' ]].mean(axis=1)
    db['std_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].std(axis=1)
    db['min_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].min(axis=1)


    db['max_p']=db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)
    db['class']=[int(s[1])-1 for s in db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)]
    db['min_in_which_fold']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)
    db['min_fold_id']=[int(s[-1]) for s in db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)]
    db['min_class']=[db['classf'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    
    for ind in range(1,5):
        db['p'+str(ind)+'_minFPro']=[db['p'+str(ind)+'f'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    

    return db

# Load 2009-2010 Data

In [5]:
# log-transformed nutrient content per 100 grams
RFFNDDS=pd.read_csv('./input_data/FNDDS_2009-10_Training_Data.csv')
RFFNDDS

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,macroclass,pythonlabel
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,-20.0,-20.0,-20.0,2047,"Salt, table",0,addition1516,2,Other,1
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,-20.0,-20.0,-20.0,11000000,"Milk, human",9602,Human milk,1,Milk,0
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,-20.0,-20.0,-20.0,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,Milk,0
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,-20.0,-20.0,-20.0,11111000,"Milk, cow's, fluid, whole",1002,"Milk, whole",1,Milk,0
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,-20.0,-20.0,-20.0,11111100,"Milk, cow's, fluid, whole, low-sodium",1002,"Milk, whole",1,Milk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94100200,"Water, bottled, sweetened, with low or no calo...",7802,Flavored or carbonated water,4,Other,3
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94100300,"Water, fruit flavored, sweetened, with high fr...",7802,Flavored or carbonated water,4,Other,3
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94210100,Propel Water,7804,Enhanced or fortified water,4,Other,3
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94210200,Glaceau Water,7804,Enhanced or fortified water,4,Other,3


In [6]:
# total nutrient list
nl=list(RFFNDDS.columns)
nl=nl[0:-7]
len(nl)

99

# Whole Nutrient Panel Analysis

In [7]:
if reproduceflag:
    model_per_foldWNP=joblib.load("./paper_classifiers_outcomes/FNDDS0910_99Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDS[RFFNDDS.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Total isoflavones']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUCWNP, performancesAUPWNP, splitsWNP, model_per_foldWNP)=AUCAUPkfold(X,y,smoteflag)


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.96997345 0.99863942 0.95228878 0.91319278]
AUC: std over the folds
[0.00369334 0.00064984 0.00437109 0.00780746]
AUP: average over the folds
[0.90305071 0.99592714 0.84712786 0.75754072]
AUP: std over the folds
[0.01670437 0.00225991 0.02011845 0.01475523]
Balanced Accuracy: average over the folds
0.8697954130393377
Balanced Accuracy: std over the folds
0.003359407638948017


In [8]:
# summary database
df=RFFNDDS.copy()
df=ClassifyDB(df,model_per_foldWNP, nl)
df.to_csv('./output_data/FNDDS_2009-10_99_Nutrients_ensemble_5folds_shuffled.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.721161,p4,3,FProf3,3,3,0.108631,0.010,0.330417,0.550952
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.590000,p4,3,FProf2,2,3,0.230000,0.070,0.290000,0.410000
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.384369,p4,3,FProf1,1,2,0.311262,0.025,0.583738,0.080000
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.915000,p4,3,FProf5,5,3,0.057500,0.010,0.045000,0.887500
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.537750,p4,3,FProf3,3,3,0.375750,0.035,0.138000,0.451250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.744000,p4,3,FProf2,2,3,0.042153,0.050,0.377694,0.530153
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.633750,p4,3,FProf3,3,3,0.177500,0.110,0.267500,0.445000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.779444,p4,3,FProf3,3,3,0.100556,0.065,0.175000,0.659444
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.616250,p4,3,FProf3,3,3,0.160000,0.185,0.262500,0.392500


# Standard Release FNDDS (for releases without flavonoids)

In [9]:
RFFNDDSSR=RFFNDDS.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
StandardNutPanel=list(RFFNDDSSR.columns)
RFFNDDSSR['pythonlabel']=RFFNDDS['pythonlabel'].copy()
print('Number of Nutrients in the Standard Nutrient Panel')
print(len(StandardNutPanel))

Number of Nutrients in the Standard Nutrient Panel
62


In [10]:
if reproduceflag:
    model_per_fold62nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_62Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSSR[RFFNDDSSR.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUC62nut, performancesAUP62nut, splits62nut, model_per_fold62nut)=AUCAUPkfold(X,y,smoteflag)

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.96341694 0.99905881 0.95588341 0.91808243]
AUC: std over the folds
[0.00266304 0.00066891 0.00390479 0.00306827]
AUP: average over the folds
[0.89113562 0.99772846 0.86810541 0.75823659]
AUP: std over the folds
[0.00891607 0.00083991 0.01486736 0.00835259]
Balanced Accuracy: average over the folds
0.8656566165843165
Balanced Accuracy: std over the folds
0.00395400739586969


In [11]:
# summary database
df=RFFNDDSSR.copy()
df=ClassifyDB(df,model_per_fold62nut,StandardNutPanel)
df.to_csv('./output_data/FNDDS_2009-10_62Nutrients_ensemble_5folds_shuffled.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.587917,p4,3,FProf5,5,2,0.155000,0.060000,0.454167,0.330833
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.650208,p4,3,FProf5,5,3,0.211667,0.046667,0.229583,0.512083
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.714643,p4,3,FProf2,2,3,0.232857,0.000000,0.105000,0.662143
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.965833,p4,3,FProf4,4,3,0.026667,0.005000,0.010000,0.958333
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.750667,p4,3,FProf1,1,3,0.149333,0.025000,0.175000,0.650667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.115000,p1,0,FProf5,5,0,0.875000,0.005000,0.015000,0.105000
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.767000,p4,3,FProf5,5,3,0.145000,0.040000,0.136000,0.679000
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.717619,p4,3,FProf3,3,3,0.170476,0.075000,0.148810,0.605714
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.801375,p4,3,FProf3,3,3,0.095000,0.051000,0.156250,0.697750


# Classify FNDDS 2015-2016 with 62 Nutrients

In [12]:
# log-transformed nutrient content per 100 grams
RFFNDDS20152016=pd.read_csv('./input_data/FNDDS_2015-16.csv')
df=ClassifyDB(RFFNDDS20152016,model_per_fold62nut,StandardNutPanel)
df.to_csv("./output_data/FNDDS_2015-16_62Nutrients_ensemble_5folds_shuffled.csv")
df

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Food code,Main food description,year,cat_digit_1,cat_digit_2,cat_digit_3,cat_digit_4,cat_digit_5,WWEIA Category number,WWEIA Category description,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,11000000,"Milk, human",2015,Milk and Milk Products,Milks and milk drinks,"Milk, human",NaN,NaN,9602,Human milk,...,0.650208,p4,3,FProf5,5,3,0.211667,0.046667,0.229583,0.512083
1,11100000,"Milk, NFS",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1004,"Milk, reduced fat",...,0.807976,p4,3,FProf2,2,3,0.145357,0.000000,0.093333,0.761310
2,11111000,"Milk, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.965833,p4,3,FProf4,4,3,0.026667,0.005000,0.010000,0.958333
3,11111100,"Milk, low sodium, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.750667,p4,3,FProf1,1,3,0.149333,0.025000,0.175000,0.650667
4,11111150,"Milk, calcium fortified, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.965833,p4,3,FProf4,4,3,0.026667,0.005000,0.010000,0.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,99991400,Cheese as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.587500,p4,3,FProf4,4,3,0.295000,0.035000,0.200000,0.470000
8686,99995000,Breading or batter as ingredient in food,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.606696,p4,3,FProf2,2,3,0.183304,0.105000,0.315000,0.396696
8687,99995130,Wheat bread as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.693333,p4,3,FProf1,1,3,0.135000,0.165000,0.178333,0.521667
8688,99998130,Sauce as ingredient in hamburgers,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.548333,p4,3,FProf3,3,3,0.300417,0.060000,0.242500,0.397083


# Nutrition Facts Analysis

In [13]:
nutrition_facts=['Protein','Total Fat','Carbohydrate','Sugars, total','Fiber, total dietary','Calcium','Iron', 'Sodium','Retinol','Carotene, beta',
 'Carotene, alpha','Cryptoxanthin, beta','Vitamin C','Cholesterol', 'Fatty acids, total saturated'];
VitaminA=['Retinol','Carotene, beta','Carotene, alpha','Cryptoxanthin, beta']
RFFNDDSNF=RFFNDDS[nutrition_facts]
RFFNDDSNF['Total Vitamin A']=RFFNDDS[VitaminA].apply(np.exp).sum(axis=1).apply(np.log)
RFFNDDSNF=RFFNDDSNF.drop(VitaminA, axis=1)
nutrition_facts=list(RFFNDDSNF.columns)
print('Number of Nutrients in the Nutrition Facts Panel')
print(len(nutrition_facts))
RFFNDDSNF['pythonlabel']=RFFNDDS['pythonlabel'].copy()

Number of Nutrients in the Nutrition Facts Panel
12


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
if reproduceflag:
    model_per_fold12nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_12Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSNF[RFFNDDSNF.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Total Vitamin A']
    y=data2train['pythonlabel']
    smoteflag=True
    (performancesAUC12nut, performancesAUP12nut, splits12nut, model_per_fold12nut)=AUCAUPkfold(X,y,smoteflag)

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.9583737  0.99674628 0.94172431 0.88474185]
AUC: std over the folds
[0.00674568 0.00196714 0.00470601 0.00913204]
AUP: average over the folds
[0.8778648  0.99166838 0.84179403 0.70668703]
AUP: std over the folds
[0.02134424 0.00403647 0.01186211 0.01687458]
Balanced Accuracy: average over the folds
0.8232695259543096
Balanced Accuracy: std over the folds
0.006712843186820426


In [15]:
# summary database
df=RFFNDDSNF.copy()
df=ClassifyDB(df,model_per_fold12nut,nutrition_facts)
df.to_csv('./output_data/FNDDS_2009-10_12Nutrients_ensemble_5folds_shuffle.csv')
df

,Protein,Total Fat,Carbohydrate,"Sugars, total","Fiber, total dietary",Calcium,Iron,Sodium,Vitamin C,Cholesterol,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.000000,-20.0,-3.729701,-8.016418,3.657337,-20.000000,-20.000000,...,0.118333,p1,0,FProf4,4,0,0.861667,0.02000,0.020000,0.098333
1,0.029559,1.477049,1.930071,1.930071,-20.0,-3.442019,-10.414313,-4.074542,-5.298317,-4.268698,...,0.515000,p4,3,FProf2,2,1,0.230000,0.30500,0.205000,0.260000
2,1.187843,0.647103,1.578979,1.625311,-20.0,-2.128632,-10.414313,-3.123566,-9.210340,-4.961845,...,0.691542,p4,3,FProf5,5,3,0.252417,0.07375,0.038333,0.635500
3,1.147402,1.178655,1.568616,1.619388,-20.0,-2.180367,-10.414313,-3.146555,-20.000000,-4.605170,...,0.933333,p4,3,FProf3,3,3,0.006667,0.10000,0.020000,0.873333
4,1.131402,1.241269,1.495149,1.495149,-20.0,-2.292635,-9.903488,-5.809143,-7.013116,-4.268698,...,0.580000,p4,3,FProf1,1,1,0.050000,0.51500,0.225000,0.210000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.000000,-20.0,-6.907755,-20.000000,-5.809143,-20.000000,-20.000000,...,0.384167,p4,3,FProf1,1,0,0.520000,0.01000,0.181667,0.288333
7250,-20.000000,-20.000000,1.504077,1.504077,-20.0,-4.074542,-20.000000,-4.828314,-4.366153,-20.000000,...,0.052500,p1,0,FProf2,2,0,0.935000,0.00000,0.025000,0.040000
7251,-20.000000,-20.000000,0.198851,0.198851,-20.0,-6.907755,-10.414313,-4.342806,-4.733004,-20.000000,...,0.492500,p3,2,FProf1,1,2,0.030000,0.00000,0.955000,0.015000
7252,-20.000000,-20.000000,1.702928,1.702928,-20.0,-4.074542,-20.000000,-20.000000,-4.366153,-20.000000,...,0.442500,p4,3,FProf2,2,0,0.430000,0.03500,0.220000,0.315000
